# T5 + doc2query - Query Expansion

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-6-3"

## Dataset Download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-04-05 22:04:11--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.67.0.90, 18.67.0.55, 18.67.0.67, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1680991452&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9u

In [ ]:
!mv corpus.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/corpus.jsonl.gz

In [ ]:
!head {main_dir}/trec-covid/corpus.jsonl

{"_id": "ug7v899j", "title": "Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia", "text": "OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60

## Libraries Installation

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.3 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.3 MB/s eta 0:00:00


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [11]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 12.4 MB/s eta 0:00:00


## Model Prediction

Initial test. Parameters documentation at https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationConfig.




In [85]:
from torch.nn.modules import padding
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained(f"{main_dir}/doc2query-adafactor-bs-32-split-1000-bf16-early-stoping")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

input_ids = tokenizer(["Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital",
                       "Surfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung",
                       "Respiratory syncytial virus (RSV) and pneumonia virus of mice (PVM) are viruses of the family Paramyxoviridae, subfamily pneumovirus, which cause clinically important respiratory infections in humans and rodents, respectively"], 
                      return_tensors="pt",
                      padding="max_length"
                       ).input_ids.to(device)

sequence_ids = model.generate(input_ids, 
                              do_sample=True,
                              num_beams=1,
                              max_length=50, num_return_sequences=3
                              )
sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
sequences

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


['which maternity ward is in the medical study of mycoplasma pneumoniae in hospital',
 'king adelaziz university hospitals mycoplasma pneumoniae',
 'Mycoplasma pneumoniae a',
 'what contributes to the innate response to impediment',
 'what does the surfactant protein-d (SP-d) do',
 'is protein D active in myoglobin',
 'respiratory syncytial virus of mice (pvm) cause',
 'respiratory syncytial virus and pneumonia virus in mice',
 'rsv pneumonia viruses of mice']

In [13]:
def predict(texts, do_sample=False, num_beams=1, num_return_sequences=1):
  input_ids = tokenizer(texts, return_tensors="pt", 
                        padding="max_length", truncation=True).input_ids.to(device)
  sequence_ids = model.generate(input_ids, do_sample=do_sample,
                                num_beams=num_beams, max_new_tokens=128, 
                                num_return_sequences=num_return_sequences)
  sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
  return sequences

In [14]:
import json

corpus = []

with open(f"{main_dir}/trec-covid/corpus.jsonl", 'r', encoding='utf-8') as f:
    # Read the file line by line
    lines = f.readlines()
    
    # Parse each line as a JSON object and store it in a list
    for line in lines:
        corpus.append(json.loads(line))


### Checking unprocessed documents

In [15]:
import jsonlines
from tqdm.auto import tqdm
import glob
import pickle
import os

In [52]:
processed_docs = set()
collections_dir = f"{main_dir}/collections/trec_covid/"
extension = "*.json"
files = glob.glob(os.path.join(collections_dir, extension))

for file in tqdm(files):
  with jsonlines.open(file) as reader:
    # Loop through each line in the file
    for item in reader:
        # Do something with the item, such as print it
        processed_docs.add(item["_id"])

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
unprocessed_corpus = [doc for doc in corpus if doc['_id'] not in processed_docs]
len(unprocessed_corpus)

### Query Expansion

In [22]:
NUM_SAMPLE_QUERIES = 3

In [25]:
import time
from collections import defaultdict
import pickle


batch_size = 8

doc_to_queries = defaultdict(list)

def add_queries(ids, queries, num_return_sequences=1):
  for i, q in enumerate(queries):
    doc_to_queries[ids[i]].append(q)
    
last_saved_batch = -1
extended_docs = []

for i in tqdm(range(0, len(unprocessed_corpus), batch_size)):
  texts = [doc["text"] for doc in unprocessed_corpus[i:i+batch_size]]
  ids = [doc["_id"] for doc in unprocessed_corpus[i:i+batch_size]]

  #Generates a batch of 1 greedy query per document
  greedy_queries = predict(texts)
  add_queries(ids, greedy_queries)
  
  #Generates a batch of 1 query per document by using beam search
  beam_search_queries = predict(texts, num_beams=2)
  add_queries(ids, beam_search_queries)
  
  #Generates 3 queries per document by using random sampling
  sampled_queries = predict(texts, do_sample=True, 
                            num_return_sequences=NUM_SAMPLE_QUERIES)
  ids_list = []
  for id in ids:
    ids_list += [id]*NUM_SAMPLE_QUERIES

  add_queries(ids_list, sampled_queries, num_return_sequences=NUM_SAMPLE_QUERIES)    


  0%|          | 0/8180 [00:00<?, ?it/s]

In [26]:
len(doc_to_queries)

65435

In [28]:
with open(f"{main_dir}/trec-covid/unjosoned_doc2queries.pickle", "wb") as f:
  pickle.dump(doc_to_queries, f)

## Indexing

### JSON serialization for indexing

In [41]:
n_files = 3
json_batch_size = len(doc_to_queries)//n_files

In [42]:
json_batch_size

21811

In [ ]:
doc_id_to_text_and_title = {doc["_id"]: (doc["text"], doc["title"]) for doc in corpus}

Creates the object to be JSON-serialized.

In [39]:
extended_docs = []

for doc_id, queries in doc_to_queries.items():
  text, title = doc_id_to_text_and_title[doc_id]
  text += '\n' + '\n'.join(queries)
  extended_docs.append({"_id":doc_id, "title":title, "text":text})

In [40]:
len(extended_docs)

65435

In [43]:
len(files)

9

In [44]:
j = len(files)

for i in tqdm(range(0, len(extended_docs), json_batch_size)):
  filename = f"{main_dir}/collections/trec_covid/json_{j}.json"
  print(filename)
  with jsonlines.open(filename, mode='w') as writer:
    for item in extended_docs[i:i+json_batch_size]:
      writer.write(item)
  j+=1

  0%|          | 0/4 [00:00<?, ?it/s]

/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_9.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_10.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_11.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_12.json


## Index building in Pyserini

Fixing some JSON serialization issues:

1.   Fields renaming
2.   Dividing the data in less JSON files of +/- the same size
3.   Removing repeated documents



In [74]:
all_docs = []
collections_dir = f"{main_dir}/collections/trec_covid/"
extension = "*.json"
files = glob.glob(os.path.join(collections_dir, extension))
doc_id_to_text = dict()

for file in tqdm(files):
  with jsonlines.open(file) as reader:
    for item in reader:
        #If the document was processed twice, choose the document with shorter 
        #text (eg.: less expanded queries - an earlier version expanded the 
        #document with 20 queries and this one uses only 5.)
        text = item["text"]
        if item["id"] in doc_id_to_text:
          other_text = doc_id_to_text[item["id"]]
          if len(other_text) <= len(text):
            text = other_text
            #Do not add this item to the list
            continue
          #Removes the previous (longer) document
          for doc in all_docs:
            if doc["id"] == item["id"]:
              all_docs.remove(doc)  
          
        all_docs.append({"id":item["id"], "contents":item["title"] + '\n' + text})
        doc_id_to_text[item["id"]] = text

  0%|          | 0/11 [00:00<?, ?it/s]

In [75]:
len(all_docs)

171332

In [76]:
assert len(all_docs) == len(corpus)

In [77]:
assert(len(doc_id_to_text)) == len(corpus)

In [78]:
json_batch_size = len(corpus)//10

In [79]:
json_batch_size

17133

In [82]:
j=0

for i in tqdm(range(0, len(all_docs), json_batch_size)):
  filename = f"{main_dir}/collections/trec_covid/json_{j}.json"
  print(filename)
  with jsonlines.open(filename, mode='w') as writer:
    for item in all_docs[i:i+json_batch_size]:
      writer.write(item)
  j+=1

  0%|          | 0/11 [00:00<?, ?it/s]

/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_0.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_1.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_2.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_3.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_4.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_5.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_6.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_7.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_8.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_9.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/json_10.json


In [83]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/ \
  --index indexes/extended_trec_covid \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-08 02:45:13,728 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-08 02:45:13,729 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-08 02:45:13,729 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-08 02:45:13,730 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid/
2023-04-08 02:45:13,730 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-08 02:45:13,730 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-08 02:45:13,731 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-08 02:45:13,731 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-04-08 02:45:

In [84]:
!mv indexes/extended_trec_covid {main_dir}